# Part 1, Topic 3, Lab A: DFA Attack Against Last MixColumns

**SUMMARY:** *Our previous fault attacks have been very implementation specific, even requiring that the compiler lay things out in a specific way. While it wouldn't be that unexpected for them to work against another target, we can actually do a fault attack that's much easier in practice. All it requires is that we insert a single byte fault between two operations near the end of AES.*

*In this lab, we'll be covering the theory behind the attack.*

##  DFA Fault Theory

There's a great article over at https://blog.quarkslab.com/differential-fault-analysis-on-white-box-aes-implementations.html, if you're interested in the actual analysis. If not, here's a quick TLDR that skips as much math as possible:

Our goal is to insert faults between the last two MixColumn operations. As a reminder, here's a block diagram of AES:

![](img/aes_operations.png)
(source: http://www.iis.ee.ethz.ch/~kgf/acacia/fig/aes.png)

This results in:

1. 2 sets of ciphertext outputs with the same plaintext - one $O$ with no errors, and one faulted output $O^\prime$
1. XORing the outputs will result in the following system of equations:
$$\space \\
O_0 + O_0^\prime = S(Y_0) + S(2Z + Y_0) \\
O_7 + O_7^\prime = S(Y_1) + S(3Z + Y_1) \\
O_{10} + O_{10}^\prime = S(Y_2) + S(Z + Y_2) \\
O_{13} + O_{13}^\prime = S(Y_3) + S(2Z + Y_3) \\
$$
1. Solving these equations will result in a set of $Y_n$ and $Z$. Here $Y_n$ is the non faulted output of Mix Columns and Add Round Key for a single column (aka the input to the final round). $Z$ is the faulted version of the first byte XORd with the non faulted version of the byte, so $aZ + Y_n$ is just the faulted version o $Y_n$. $S(x)$ is the SBox operation, $+$ is an XOR, and multiplications are done in $GF(2^8)$ (we've got a special `gmul()` function to do this for us).
1. $Y_n$ is constant between faults with the same plaintext (it's only made up of non-faulted bytes so faults have no effect on it) - another fault is enough to narrow $Y_n$ down to one value per byte
1. $Y_n$ can then be turned into 4 key bytes with the following equations:
$$\space\\
\begin{aligned}
K_{10,0} &=S\left(Y_{0}\right)+O_{0} \\
K_{10,7} &=S\left(Y_{1}\right)+ O_{7}\\
K_{10,10} &=S\left(Y_{2}\right)+ O_{10}\\
K_{10,13} &=S\left(Y_{3}\right)+O_{13}
\end{aligned}$$


The first system of equations is non-linear with multiple solutions, so it's going to be much easier to just brute force it -  aka try every possible $Z$, $Y_0$, $Y_1$, $Y_2$, and $Y_3$ value in these equations, taking only the ones that work for all the equations. You can make this much faster by short circuiting - as soon as it fails one of these equations, there's no need to continue on from that point. For example, if you're going through the equations in the above sequence and the second one fails, there's no need to continue on with $Y_2$ and $Y_3$ for that particular combination of $Z$, $Y_0$, and $Y_1$.

We can test this theory by stopping AES just before the 9th round mix columns and changing the 0th byte to a random value. ChipWhisperer includes an AES Cipher that we can use to do the encryption.

In [ ]:
# get an AES cipher
import chipwhisperer as cw
from chipwhisperer.common.utils.aes_cipher import AESCipher, aes_tables
import chipwhisperer.analyzer as cwa
ktp = cw.ktp.Basic()
key = list(ktp.next()[0])
for i in range(10):
    key.extend(cwa.aes_funcs.key_schedule_rounds(key[0:16], 0, i+1))
    
cipher = AESCipher(key)
print(bytearray(key))

Just to verify that our cipher works, let's use Pycryptodome to also do an encryption. We can then compare the results to make sure they match.

In [ ]:
# verify that our cipher works
from Crypto.Cipher import AES
check_cipher = AES.new(ktp.next()[0], AES.MODE_ECB)

#generate random plaintext
pt = ktp.next()[1]

#encrypt with both
ct1 = cipher.cipher_block(list(pt))
ct2 = check_cipher.encrypt(pt)

# verify that outputs are the same
bytearray(ct1) == bytearray(ct2)

We can generate our glitch by doing the AES encryption, stopping at the 9th round mix columns, and changing a value, then completing the encryption. We also make a copy of the state before we insert the fault and complete AES on that as well.

In [ ]:
import random

def generate_glitch(pt, cipher):
    # Do AES, but stop before the last Mix Columns
    state = list(pt)
    state = state+[16-len(state)]*(16-len(state))
    cipher._add_round_key(state, 0)
    for i in range(1, 9):
        cipher._sub_bytes(state)
        cipher._shift_rows(state)
        cipher._mix_columns(state, False)
        cipher._add_round_key(state, i)
    cipher._sub_bytes(state)
    cipher._shift_rows(state)

    # Make a copy of the state and finish the rest of AES
    x = list(state)
    cipher._mix_columns(x, False)
    cipher._add_round_key(x, 9)
    cipher._sub_bytes(x)
    cipher._shift_rows(x)
    cipher._add_round_key(x, 10)

    # Insert a fault and go through the rest of AES with the fault
    import random
    random.seed()
    fault = random.getrandbits(8)
    if state[0] == fault:
        fault += 1
        fault = fault % 0xFF
    
    state[0] = fault
    cipher._mix_columns(state, False)
    cipher._add_round_key(state, 9)
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    cipher._add_round_key(state, 10)
    return state, x


Let's generate some good and faulty output:

In [ ]:
# uncomment to generate a random plaintext
#pt = ktp.next()[1] 

#get two outputs, a normal and a faulty one
O_fault, O_good = generate_glitch(pt, cipher) 

#should be the same except for bytes 0, 7, 10, 13
print(bytearray(O_fault), bytearray(O_good)) 

Here's our code to brute force the Y values, update our Y's with new Y values, and convert a Y value into a key. `gmul()` performs a multiply over $GF(2^8)$, and `check_Y()` checks that a given Y and Z fulfill the requirements of our system of equations.

In [ ]:

from tqdm.notebook import trange
def get_Y_guesses(state, x):
    #GF(2^8) multiplication adapted from https://en.wikipedia.org/wiki/Finite_field_arithmetic#C_programming_example
    def gmul(a, b): 
        p = 0
        while a and b:
            if b & 1:
                p ^= a
            if (a & 0x80):
                a = (a << 1) ^ 0x11b;
            else:
                a <<= 1
            b >>= 1
        return p
    
    #check that Yn and Z fulfill requirements
    def check_Y(Z, Yn, n):
        lookup = [0, 7, 10, 13]
        lhs = state[lookup[n]] ^ x[lookup[n]]
        coeff = [2, 3, 1, 1]
        rhs = aes_tables.sbox[Yn] ^ aes_tables.sbox[gmul(Z, coeff[n]) ^ Yn]
        return lhs == rhs
    guesses = []
    
    # brute force Z and Yn
    for Z in trange(255):
        for Y0 in range(255):
            if check_Y(Z, Y0, 0):
                for Y1 in range(255):
                    if check_Y(Z, Y1, 1):
                        for Y2 in range(255):
                            if check_Y(Z, Y2, 2):
                                for Y3 in range(255):
                                    if check_Y(Z, Y3, 3):
                                        guesses.append((Y0, Y1, Y2, Y3))
    return guesses
    
def update_Y_guesses(Y_old, Y_new):
    updated_Y = []
    for Ys in Y_old:
        if Ys in Y_new:
            updated_Y.append(Ys)
    return updated_Y

def Y_to_key(x, Y):
    return aes_tables.sbox[Y] ^ x

We can brute force the Y values:

In [ ]:
# calculate Y_n for the outputs
Y_guesses = get_Y_guesses(O_fault, O_good) 
#print(Y_guesses)

Then generate a new fault, brute force Y values, and find the ones that match between the faults:

In [ ]:
#get a new fault with the same plaintext (O_good will be the same)
O_fault, O_good = generate_glitch(pt, cipher) 
print(bytearray(O_fault), bytearray(O_good)) 

# update our Y values with ones that also work for the new fault
Y_guesses = update_Y_guesses(Y_guesses, get_Y_guesses(O_fault, O_good)) 

#should be left with one Y guess per
print(Y_guesses)

We can then print the key bytes that we recovered, as well as the bytes for the actual last round key:

In [ ]:
#turn Y_n into key bytes
lookup = [0, 7, 10, 13]
print("bytes recovered: ", bytearray([Y_to_key(O_good[lookup[n]], Y_guesses[0][n]) for n in range(4)]))
print("key bytes:       ", bytearray([key[160:][i] for i in lookup]))

## Extending the Attack

There's a lot of scenarios that we didn't cover at all here:

1. We only attacked one column of AES (aka 4 key bytes). A full attack will need to attack the rest of the columns as well, with the output lookup needing to change for each column.
    * If we fault a single column of the previous round MixColumns, this will actually turn into a single byte fault for each column in the next round! MixColumns will spread the fault to each byte in the column. Each byte in the column is then placed in a separate column by the next ShiftRows.
1. We assumed the fault was inserted in the first byte of the column. If the glitch was inserted at another byte in that column, the system of equations we solved changes. For a real attack, we don't know which byte we glitched, so we'd need to account for that in the attack. Depending on the implementation, you might also glitch multiple bytes in the same column, which you'd have to discard.
1. We only did AES128 here. If this was AES256, we'd need to do the attack again for the previous round as well.

Let's try updating the attack to also work if we fault a random byte in the column. We can update our function that brute forces the Y values to also take a fault_byte argument. Z is the only part that changes (remember, Y doesn't depend on anything to do with the fault!), so we can update our coefficient table to take a fault_byte argument as well.

## Attacking Other Bytes

If we want to take the other bytes, we'll actually have to look a bit more into the math (or not, feel free to skip this section if you don't care where these updated coefficients are coming from). If the AES column state is:

$$\left(\begin{array}{llll}
A & E & I & M \\
B & F & J & N \\
C & G & K & O \\
D & H & L & P
\end{array}\right)$$

then $Y_n$ looks like:

$$
Y_{0}=2 A+3 B+C+D+K_{9,0} \\
Y_{1}=3 A+B+C+2 D+K_{9,3} \\
Y_{2}=A+B+2 C+3 D+K_{9,2} \\
Y_{3}=A+2 B+3 C+D+K_{9,1}
$$

For the byte A attack, we needed to make the coefficient the same as the one in front of A for those equations. For the other bytes, we just need use the coefficient for that byte. For example, for B, instead of `[2, 3, 1, 1]`, we need to use `[3, 1, 1, 2]`.

In [ ]:
def get_Y_guesses(state, x, fault_byte):
    #GF(2^8) multiplication adapted from https://en.wikipedia.org/wiki/Finite_field_arithmetic#C_programming_example
    def gmul(a, b): 
        p = 0
        while a and b:
            if b & 1:
                p ^= a
            if (a & 0x80):
                a = (a << 1) ^ 0x11b;
            else:
                a <<= 1
            b >>= 1
        return p
    
    #check that Yn and Z fulfill requirements
    def check_Y(Z, Yn, n, fault_byte):
        lookup = [0, 7, 10, 13]
        lhs = state[lookup[n]] ^ x[lookup[n]]
        coeff = [[2, 3, 1, 1], [3, 1, 1, 2], [1, 1, 2, 3], [1, 2, 3, 1]]
        rhs = aes_tables.sbox[Yn] ^ aes_tables.sbox[gmul(Z, coeff[fault_byte][n]) ^ Yn]
        return lhs == rhs
    guesses = []
    
    # brute force Z and Yn
    for Z in trange(255):
        for Y0 in range(255):
            if check_Y(Z, Y0, 0, fault_byte):
                for Y1 in range(255):
                    if check_Y(Z, Y1, 1, fault_byte):
                        for Y2 in range(255):
                            if check_Y(Z, Y2, 2, fault_byte):
                                for Y3 in range(255):
                                    if check_Y(Z, Y3, 3, fault_byte):
                                        guesses.append((Y0, Y1, Y2, Y3, fault_byte))
    return guesses

In [1]:
def gmul(a, b): 
    p = 0
    while a and b:
        if b & 1:
            p ^= a
        if (a & 0x80):
            a = (a << 1) ^ 0x11b;
        else:
            a <<= 1
        b >>= 1
    return p

In [8]:
a = 0x60
b = gmul(a, 2) ^ gmul(a, 3) ^ a ^ a

In [9]:
hex(b)

'0x60'

We also need to update our glitch generation to randomly insert the glitch:

In [ ]:
def generate_glitch(pt, cipher):
    # Do AES, but stop before the last mix columns
    state = list(pt)
    state = state+[16-len(state)]*(16-len(state))
    cipher._add_round_key(state, 0)
    for i in range(1, 9):
        cipher._sub_bytes(state)
        cipher._shift_rows(state)
        cipher._mix_columns(state, False)
        cipher._add_round_key(state, i)
    cipher._sub_bytes(state)
    cipher._shift_rows(state)

    # make a copy of the state and run it through the rest of AES
    x = list(state)
    cipher._mix_columns(x, False)
    cipher._add_round_key(x, 9)
    cipher._sub_bytes(x)
    cipher._shift_rows(x)
    cipher._add_round_key(x, 10)

    # insert a random fault byte in a random location
    import random
    random.seed()
    fault = random.getrandbits(8)
    fault_byte = random.getrandbits(2)
    if state[fault_byte] == fault:
        fault += 1
        fault = fault % 0xFF
    state[fault_byte] = fault

    #and take the faulted one through AES as well
    cipher._mix_columns(state, False)
    cipher._add_round_key(state, 9)
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    cipher._add_round_key(state, 10)
    return state, x

Then we can just repeat what we did before:

In [ ]:
#generate a random plaintext
pt = ktp.next()[1] 

#get two outputs, a normal and a faulty one
O_fault, O_good = generate_glitch(pt, cipher) 

#should be the same except for bytes 0, 7, 10, 13
print(bytearray(O_fault), bytearray(O_good)) 

And do the Y value brute force for each byte in the column.

In [ ]:
Y_guesses = []
for fault_byte in range(4):
    Y_guesses.extend(get_Y_guesses(O_fault, O_good, fault_byte))
    
print(Y_guesses)

Our Y update function also has to change, since each fault could have a different byte (and we appended which byte was faulted to the Y values). We can also take the opportunity to print which bytes were faulted:

In [ ]:
def update_Y_guesses(Y_old, Y_new):
    updated_Y = []
    for Ys in Y_old:
        for Ys_new in Y_new:
            if Ys[:-1] == Ys_new[:-1]:
                updated_Y.append(Ys[:-1])
                print("Fault in bytes {} and {}".format(Ys[-1], Ys_new[-1]))
    return updated_Y

The rest is pretty similar to before. Generate a new glitch, brute force Y values, then match them to the old Y values.

In [ ]:
#get a new fault with the same plaintext (O_good will be the same)
O_fault, O_good = generate_glitch(pt, cipher) 
print(bytearray(O_fault), bytearray(O_good)) 

new_Y = []
for fault_byte in range(4):
    new_Y.extend(get_Y_guesses(O_fault, O_good, fault_byte))
# update our Y values with ones that also work for the new fault
Y_guesses = update_Y_guesses(Y_guesses, new_Y) 

#should be left with one Y guess per
print(Y_guesses) 



As you can see, only one combination of byte faults will result in a match: we only need 2 faults, even if we don't know which bytes were faulted!

In [ ]:
#turn Y_n into key bytes
lookup = [0, 7, 10, 13]
print("bytes recovered: ", bytearray([Y_to_key(O_good[lookup[n]], Y_guesses[0][n]) for n in range(4)]))
print("key bytes:       ", bytearray([key[160:][i] for i in lookup]))

Here's everything together. Try running this block a few times to get glitches with different combinations of bytes.

In [ ]:
#generate a random plaintext
pt = ktp.next()[1] 

#get two outputs, a normal and a faulty one
O_fault, O_good = generate_glitch(pt, cipher) 

#should be the same except for bytes 0, 7, 10, 13
print(bytearray(O_fault), bytearray(O_good)) 

Y_guesses = []
for fault_byte in range(4):
    Y_guesses.extend(get_Y_guesses(O_fault, O_good, fault_byte))

#get a new fault with the same plaintext (O_good will be the same)
O_fault, O_good = generate_glitch(pt, cipher) 
print(bytearray(O_fault), bytearray(O_good)) 

new_Y = []
for fault_byte in range(4):
    new_Y.extend(get_Y_guesses(O_fault, O_good, fault_byte))
# update our Y values with ones that also work for the new fault
Y_guesses = update_Y_guesses(Y_guesses, new_Y) 

#should be left with one Y guess per
print(Y_guesses)

#turn Y_n into key bytes
lookup = [0, 7, 10, 13]
print("bytes recovered: ", bytearray([Y_to_key(O_good[lookup[n]], Y_guesses[0][n]) for n in range(4)]))
print("key bytes:       ", bytearray([key[160:][i] for i in lookup]))

## Conclusions and Next Steps

Compared to our earlier attacks, this attack is much more applicable to real hardware. Really, the only requirement besides being able to repeat the encryption operation with the same plaintext, being able to observe the ciphertext, and being able to introduce a single byte fault in a column (or a multi byte fault in a single column if we fault the 8th round instead). Given these requirements, it's possible to fault any implementation of AES.

In the next tutorial, we'll look at doing this attack on real hardware and utilizing a library to do the analysis.